In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks_cwt
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics.pairwise import pairwise_distances

In [1]:
class ILS():
    
    def __init__(self, n_clusters = None, min_cluster_size = None, metric = 'euclidean'):
        
        self.n_clusters = n_clusters # need to calculate defaults based on data set input
        self.min_cluster_size = min_cluster_size
        self.metric = metric
        
    def fit(self, X):
        
        if self.min_cluster_size is None: # added just so that it will run, but need to decide on better default
            self.min_cluster_size = int (0.05 * X.shape[0])
        
        self.data_set = np.concatenate((X.to_numpy(), np.zeros((X.shape[0],))))
        self.rmin = []
        
        X = np.array(X)
        unlabelled = [i + 1 for i in range(X.shape[0] - 1)] # step 1
        label_spreading = self.label_spreading([0], unlabelled_points)
        
        new_centers, new_unlabelled = find_initial_points() # step 2
        
        label_spreading = self.label_spreading(new_centers, new_unlabelled) #step 3
    
    def find_minima(self):
        '''
        Written by Amanda Parker
        Given the index of the peaks used for partitioning the dataset into cluster find the point of maximum density
        OUTPUT:
            index = list index of r_min plot 
        '''
        
        if self.rmin is []:
            raise Exception("ILS has not been run yet")
        
        filtered = guassian_filter1d(self.rmin, self.min_cluster_size) #smooth rmin
        index = np.arange(len(filtered))

        maxima = find_peaks_cwt(filtered, len(filtered) * [self.min_cluster_size])
        maxima = [i for i in maxima if i < len(filtered) - self.min_cluster_size]
        maxima = [i for i in maxima if i >  window] #removing peaks at the beginning and end
        
        betweenMax = np.split(filtered, maxima)
        betweenIndex = np.split(index, maxima)

        minVal = [min(i) for i in betweenMax]
        subMinIndex = [np.argmin(i) for i in betweenMax]
        minima = [betweenIndex[i][subMinIndex[i]] for i in range(len(subMinIndex))]
        minima = [i for i in minima if i != 0]
        
        self.n_clusters = len(minima)
        
        return minima
    
    def find_initial_points(self):
        
        labelled_points = self.find_minima()
        
        counter = 1
        
        for i in labelled_points: # assign labels as integers each with different label
            self.data_set[i, -1] = counter
            counter += 1
        
        unlabelled_points = [i for i in range(len(labelled_points)) if not i in labelled_points]
                 
        return labelled_points, unlabelled_points
    
    def label_spreading(self, labelled_points, unlabelled_points):
        '''
        Written by Amanda Parker
        Applies iterative label spreading to the given points
        INPUTS:
            labelled_points = initial points that are already labelled
                2D array with last column the points label
            unlabelled_points = points in the data set that are not labelled
                2D array with last column the points label (0)
        OUTPUTS:
            r_min = an 1D array which contains the R_min distance for eeach iteration
        '''
        featureColumns = self.data_set[0, :self.data_set.shape[1]] # Keep original index columns in DF
        indices = np.arange(self.data_set.shape[0]) 
        oldIndex = np.arange(self.data_set.shape[0]) 
       
        labelled = self.data_set[labelled_points]
        unlabelled = self.data_set[unlabelled_points]
        labelColumn = self.data_set.shape[1]-1
        # lists for ordered output data
        outD = []
        outID = []
        closeID = []
    
        # Continue to label data points until all data points are labelled
        while len(unlabelled) > 0 :
            # Calculate labelled to unlabelled distances matrix (D) 
            D = pairwise_distances(
                labelled,
                inlabelled)
            # Find the minimum distance between a labelled and unlabelled point
            # first the argument in the D matrix
            (posL, posUnL) = np.unravel_index(D.argmin(), D.shape)
            
            # Switch label from 0 to new label
            unlabelled[posUnL, labelColumn] = labelled[posL,labelColumn] 
            # move newly labelled point to labelled dataframe
            labelled = np.concatenate((labelled, unlabelled[posUnL]), axis=None)
            # drop from unlabelled data frame
            unlabelled = np.delete(unlabelled, idUnL, 0)
            
            # output the distance and id of the newly labelled point
            outD.append(D.min())
            outID.append(posUnL)
            closeID.append(posL)
            
        # Throw error if loose or duplicate points
        if len(labelled) + len(unlabelled) != self.data_set.shape[0] : 
            raise Exception(
                '''The number of labelled ({}) and unlabelled ({}) points does not sum to the total ({})'''.format( len(labelled), len(unlabelled),self.data_set.shape[0]) )
        # Reodered index for consistancy
        newIndex = oldIndex[outID]
        # Column 1 = indices of point in orginal dataset, Column 2 = corresponding Rmin
        orderLabelled = np.concatenate((newIndex, outD), axis=1) 

        # ID of point label was spread from
        closest = np.concatenate((newIndex, closeID), axis=1)

        newLabels = labelled[:,self.data_set.shape[1]-1]
        # return
        return newLabels, np.concatenate((orderLabelled, closest), axis=1)